In [1]:
import re
import os 
import sys 

import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io

from pathlib import Path
from tqdm.notebook import trange, tqdm
from joblib import Parallel, delayed
from skimage import exposure
import h5py
import pandas as pd
import scanpy as sc
import squidpy as sq
sc.settings.verbosity = 3

from matplotlib.pyplot import rc_context
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from functools import reduce
from matplotlib import cm, colors
import scanorama
import seaborn as sns 
import anndata as ad
from PIL import Image

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)
    

In [4]:
from config import *


In [5]:
data_ROI = data_dir / 'ROI_new'
h5_data = data_ROI / f'TMA.hdf5'
with h5py.File(h5_data,  "a") as f:
    print(list(f.keys()))
    # del f['A1']

['B7', 'C3', 'D1', 'D2', 'D5', 'D9', 'F7', 'G3', 'H1', 'H2', 'H5', 'H9']


# Read info

In [6]:
from skimage.transform import rotate
from functools import partial
import matplotlib.patches as mpatches
from skimage.segmentation import mark_boundaries
from skimage.filters import median
from skimage.morphology import disk
import skimage.io

def get_info(img_folder):
    """Function returns the info from folder containing multi-cycle staigning on cell

    Args:
        img_folder (str) : imgage folder path to get information
        name_dict (dict) : three level dictionnary mapping cycle -> channel -> marker name

    Returns:
        pandas dataframe with information
    """
    images_path = []
    markers = []
    rois = []
    
    # Loop through image folder
    for (dirpath, dirnames, filenames) in os.walk(img_folder):
        for name in sorted(filenames):
            if 'ome.tiff' not in name:
                continue 
                
            roi = dirpath.split('_')[-1]
            try:
                marker = name.split('_')[2].split('.')[0]
                if marker == 'contaminant':
                    continue
                elif marker == 'DNA':
                    if '191Ir' in name:
                        marker += '1'
                    else:
                        marker += '2'
            except:continue
            
            path = os.path.join(dirpath, name)
            rois.append(roi)
            markers.append(marker)
            images_path.append(path)
            
    info = {
        "ROI": rois,
        "Marker": markers,
        "Path": images_path,
    }
    df = pd.DataFrame(info)
    return df


In [7]:
data_ROI = data_dir / 'ROI_new'
df = get_info(data_ROI)


In [8]:
df = df[~df.Marker.isin(['DNA1', 'DNA2'])]
df.to_csv(data_ROI / 'info.csv', index=False)

In [9]:
df.Marker.unique()

array(['SMA', 'Vimentin', 'TCF1', 'CD163', 'Pankeratin', 'H3K9me3',
       'PDL1', 'CD103', 'CD206', 'CD11c', 'FoxP3', 'CD4', 'E-cadherin',
       'CD68', 'CD95', 'CD20', 'CD8', 'PD1', 'GranenzymeB', 'Ki67',
       'COL1', 'CD3', 'Histone3', 'CD45Ro', 'HLADR', 'MHC-II'],
      dtype=object)

# Read images, process and save to h5

In [10]:
from sklearn.neighbors import NearestNeighbors
from skimage.util import img_as_ubyte 

def contrast_streching(img):
    p2, p98 = np.percentile(img, (30,99))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

def read_img(path:str) -> np.ndarray:
    '''
    Read image from path
    '''
    img = io.imread(path, as_gray=True)
    img = contrast_streching(img)
    img = img_as_ubyte(img)
    return img

def joblib_loop(task, pics):
    return Parallel(n_jobs=20)(delayed(task)(i) for i in pics)

def get_NN(data, n):
    fit = NearestNeighbors(n_neighbors=n).fit(data)
    distances, indices = fit.kneighbors(data)

    return distances, indices

def filter_img_knn(img, n=25, th=3.5):
    # Get avg distances per positive expressed pixels
    x, y = np.where(img > 0)
    values = img[x,y]
    
    data = np.column_stack((x,y))
    distances, indices = get_NN(data, n)
    # avg_dist = np.average(distances, axis=1, weights=values[indices])
    avg_dist = np.average(distances, axis=1)
        
    filter_ind = avg_dist > th
    unique, counts = np.unique(filter_ind, return_counts=True)
    print(unique, counts)
    x_fil = x[filter_ind]
    y_fil = y[filter_ind]

    img_fil = img.copy()
    img_fil[x_fil, y_fil] = 0
    
    return img_fil

def save_hdf5(path:str, name:str, data: np.ndarray, attr_dict= None, mode:str='a') -> None:
    # Read h5 file
    hf = h5py.File(path, mode)
    # Create z_stack_dataset
    if hf.get(name) is None:
        data_shape = data.shape
        data_type = data.dtype
        chunk_shape = (1, ) + data_shape[1:]
        max_shape = (data_shape[0], ) + data_shape[1:]
        dset = hf.create_dataset(name, shape=data_shape, maxshape=max_shape, chunks=chunk_shape, dtype=data_type, compression="gzip")
        dset[:] = data
        if attr_dict is not None:
            for attr_key, attr_val in attr_dict.items():
                dset.attrs[attr_key] = attr_val
    else:
        print(f'Dataset {name} exists')
        
    hf.close()

In [11]:
group = df.groupby('ROI')
h5_data = data_ROI / f'TMA.hdf5'

for name, df_group in group:
    with h5py.File(h5_data,  "r") as f:
        if name in f:
            print(f'Dataset {name} exists')
            continue
    
    # Read images
    paths = df_group.Path.tolist()
    imgs = joblib_loop(read_img, paths)
    imgs = joblib_loop(filter_img_knn, imgs)
    imgs=np.stack(imgs, axis=0)
    # read markeabs    
    markers = df_group.Marker.tolist()
    print(len(markers))
    #Save h5
    save_hdf5(h5_data, name, imgs, {'labels': markers})


Dataset B7 exists
Dataset C3 exists
Dataset D1 exists
Dataset D2 exists
Dataset D5 exists
Dataset D9 exists
Dataset F7 exists
Dataset G3 exists
Dataset H1 exists
Dataset H2 exists
Dataset H5 exists
Dataset H9 exists


# Save to adata file

In [12]:
data_ROI = data_dir / 'ROI_new'

def get_masks(mask_folder):
    '''
    Function to get all mask from mask forlder
    '''
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tif" in name:
                filename = os.path.join(dirpath, name)
                img = skimage.io.imread(filename)
                condition = name.split('_')[1][:2]
                masks[condition] = img
            else:
                continue
    return masks

def read_intensity_per_cell(img, mask):
    props = skimage.measure.regionprops_table(mask, img,
                                             properties = ['label', 'mean_intensity', 'area'] )
    df_prop = pd.DataFrame(props)
    df_prop['mean_intensity'] = df_prop['mean_intensity']
    df_prop.drop('area', axis=1, inplace=True)
        
    # x_scaled = MinMaxScaler().fit_transform(df_prop[['mean_intensity']])
    # df_prop['mean_intensity'] = x_scaled
    
    return df_prop

def get_imgs(file_path, name):
    f = h5py.File(file_path, 'r')
    imgs = f[name]
    labels = list(f[name].attrs['labels'])
    return imgs, labels

In [13]:
h5_data = data_ROI / f'TMA.hdf5'
df = pd.read_csv(data_ROI / f"info.csv")
# markers_subset = ['SMA', 'Vimentin', 'Pankeratin', 'E-cadherin', 'COL1', 'CD163', 'CD206', 'CD68', 'HLADR', 'TCF1', 'CD103', 'CD8', 'TCF1', 'CD103', 'CD4', 'CD3']
# markers_subset = ['SMA', 'Pankeratin', 'E-cadherin', 'COL1', 'FoxP3', 'CD163', 'CD206', 'CD68', 'HLADR', 'TCF1', 'CD103', 'CD8', 'CD4', 'CD3', 'CD45Ro', 'PDL1', 'Ki67']
markers_subset = ['SMA', 'Vimentin', 'TCF1', 'CD163', 'Pankeratin', 'H3K9me3',
       'PDL1', 'CD103', 'CD206', 'CD11c', 'FoxP3', 'CD4', 'E-cadherin',
       'CD68', 'CD95', 'CD20', 'CD8', 'PD1', 'GranenzymeB', 'Ki67',
       'COL1', 'CD3', 'CD45Ro', 'HLADR', 'MHC-II']

ROIs = df.ROI.unique()
masks = get_masks(data_ROI / 'masks')

df_all = []
centroids = []

for roi in tqdm(ROIs, total=len(ROIs)):
    imgs, markers = get_imgs(h5_data, str(roi))
    mask = masks[str(roi)]
    
    df_appended_list = []
    for i, img in enumerate(imgs):
        if markers[i] in markers_subset:
            df_prop = read_intensity_per_cell(img, mask)
            df_prop.columns = ['Cell_label', markers[i]]
            df_appended_list.append(df_prop)
        
    # Combine dataframe
    df_cell_intensity = reduce(lambda left,right: pd.merge(left,right,on=['Cell_label']), df_appended_list)
    df_cell_intensity['ROI'] = roi
    df_all.append(df_cell_intensity)

    props = skimage.measure.regionprops_table(mask, properties = ['label', 'centroid'] )
    rows = props['centroid-0'] 
    cols = props['centroid-1']
    centroid = np.array(list(zip(cols,rows)))
    centroids.append(centroid)


centroids = np.concatenate(centroids)

  0%|          | 0/12 [00:00<?, ?it/s]

In [14]:
df = pd.concat(df_all, ignore_index=True)
df.to_csv(data_ROI/ "cell_exp_.csv", index=False)
df.head()

,Cell_label,SMA,Vimentin,TCF1,CD163,Pankeratin,H3K9me3,PDL1,CD103,CD206,...,CD8,PD1,GranenzymeB,Ki67,COL1,CD3,CD45Ro,HLADR,MHC-II,ROI
0,1,0.000000,2.644444,0.0,0.0,0.0,11.311111,0.000000,0.0,0.0,...,2.211111,0.0,0.0,0.0,0.000000,0.000000,5.955556,6.444444,3.577778,B7
1,2,1.552381,0.666667,0.0,0.0,0.0,32.580952,19.876190,0.0,0.0,...,12.704762,0.0,0.0,0.0,0.000000,16.876190,17.504762,8.771429,7.580952,B7
2,3,1.694444,0.000000,0.0,0.0,0.0,35.314815,12.314815,0.0,0.0,...,6.620370,0.0,0.0,0.0,0.157407,0.583333,23.351852,13.694444,11.555556,B7
3,4,0.000000,0.000000,0.0,0.0,0.0,6.781250,3.968750,0.0,0.0,...,3.406250,0.0,0.0,0.0,0.000000,5.906250,24.000000,21.281250,12.312500,B7
4,5,0.000000,0.000000,0.0,0.0,0.0,9.633333,19.000000,0.0,0.0,...,20.633333,0.0,0.0,0.0,0.000000,0.000000,17.166667,21.433333,11.733333,B7


In [15]:
df_intensity = df.iloc[:,1:-1]
df_intensity.head()

,SMA,Vimentin,TCF1,CD163,Pankeratin,H3K9me3,PDL1,CD103,CD206,CD11c,...,CD20,CD8,PD1,GranenzymeB,Ki67,COL1,CD3,CD45Ro,HLADR,MHC-II
0,0.000000,2.644444,0.0,0.0,0.0,11.311111,0.000000,0.0,0.0,0.000000,...,0.0,2.211111,0.0,0.0,0.0,0.000000,0.000000,5.955556,6.444444,3.577778
1,1.552381,0.666667,0.0,0.0,0.0,32.580952,19.876190,0.0,0.0,4.857143,...,0.0,12.704762,0.0,0.0,0.0,0.000000,16.876190,17.504762,8.771429,7.580952
2,1.694444,0.000000,0.0,0.0,0.0,35.314815,12.314815,0.0,0.0,11.333333,...,0.0,6.620370,0.0,0.0,0.0,0.157407,0.583333,23.351852,13.694444,11.555556
3,0.000000,0.000000,0.0,0.0,0.0,6.781250,3.968750,0.0,0.0,7.968750,...,0.0,3.406250,0.0,0.0,0.0,0.000000,5.906250,24.000000,21.281250,12.312500
4,0.000000,0.000000,0.0,0.0,0.0,9.633333,19.000000,0.0,0.0,1.700000,...,0.0,20.633333,0.0,0.0,0.0,0.000000,0.000000,17.166667,21.433333,11.733333


In [16]:
df.ROI.unique()

array(['B7', 'F7', 'C3', 'D1', 'D2', 'D5', 'D9', 'G3', 'H1', 'H2', 'H5',
       'H9'], dtype=object)

In [17]:
adata_path = data_ROI  / f"raw_Tcell_subset.h5ad"

In [18]:
if os.path.exists(adata_path):
    adata = ad.read_h5ad(adata_path)
else:
    # Create annData from dataframe
    adata = sc.AnnData(df_intensity.values)
    adata.var_names = df_intensity.columns.tolist() # add variable name

    # Add obs information
    adata.obs['Cell'] = df.Cell_label.tolist()
    adata.obs['ROI'] = df.ROI.tolist() 
    adata.obsm['spatial'] = centroids
    adata.write(adata_path)  

C:\Users\thu71\Anaconda3\envs\new_codex\lib\site-packages\anndata\_core\anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'ROI' as categorical


In [19]:
# Combat
adatas  = []
for batch in adata.obs["ROI"].unique():
    adata_subset = adata[
        adata.obs["ROI"] == batch,
    ]
    sc.pp.scale(adata_subset, max_value=2.5)
    adatas.append(adata_subset)

adata = ad.concat(adatas)
adata.raw = adata
sc.pp.combat(adata, key='ROI')
    

C:\Users\thu71\Anaconda3\envs\new_codex\lib\site-packages\scanpy\preprocessing\_simple.py:843: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)
C:\Users\thu71\Anaconda3\envs\new_codex\lib\site-packages\anndata\_core\anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'ROI' as categorical


Standardizing Data across genes.

Found 12 batches

Found 0 numerical variables:
	

Fitting L/S model and finding priors

Finding parametric adjustments

Adjusting data



In [20]:
adata_path = data_ROI  / f"raw_scanorama_subset.h5ad"

alldata = {}
for batch in adata.obs['ROI'].unique():
    adata_subset = adata[adata.obs['ROI'] == batch,]
    alldata[batch] = adata_subset
    
#convert to list of AnnData objects
adatas = list(alldata.values())

# run scanorama.integrate
scanorama.integrate_scanpy(adatas, sketch=True)

# make into one matrix.
adata_integrated = ad.concat(adatas)
print(adata.shape)
print(adata_integrated.shape)

Found 17 genes among all datasets
[[0.     0.4941 0.5633 0.5832 0.5557 0.4228 0.406  0.5293 0.2531 0.4655
  0.4081 0.3517]
 [0.     0.     0.2384 0.2946 0.5903 0.4011 0.6985 0.2275 0.2222 0.1817
  0.2317 0.6329]
 [0.     0.     0.     0.5606 0.403  0.2834 0.2983 0.7316 0.1762 0.4932
  0.213  0.1891]
 [0.     0.     0.     0.     0.4973 0.4672 0.3051 0.6022 0.4475 0.3986
  0.3439 0.2219]
 [0.     0.     0.     0.     0.     0.5606 0.486  0.4316 0.3204 0.5233
  0.4435 0.5391]
 [0.     0.     0.     0.     0.     0.     0.444  0.3245 0.5884 0.3046
  0.8033 0.4817]
 [0.     0.     0.     0.     0.     0.     0.     0.3586 0.2372 0.2899
  0.2883 0.8027]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.4213 0.6788
  0.4297 0.3265]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.3332
  0.564  0.2567]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.4745 0.4231]
 [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.4

In [21]:
adata_integrated.write(adata_path)

C:\Users\thu71\Anaconda3\envs\new_codex\lib\site-packages\anndata\_core\anndata.py:1228: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Reordering categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'ROI' as categorical


In [22]:
adata_integrated.var_names

Index(['SMA', 'TCF1', 'CD163', 'Pankeratin', 'PDL1', 'CD103', 'CD206', 'FoxP3',
       'CD4', 'E-cadherin', 'CD68', 'CD8', 'Ki67', 'COL1', 'CD3', 'CD45Ro',
       'HLADR'],
      dtype='object')